# Convert_Coordinate
したいこと  
→HinodeのCSVデータの座標系がHelioprojectiveでSHARPデータセットの座標系がHeliographic Stonyhurstなのでこの座標系を統一して変換可能な形にしたい  
## 参考
[Sunpy-coordinate](https://docs.sunpy.org/en/0.7/code_ref/coordinates.html)

In [3]:
# Example
from astropy.wcs.utils import wcs_to_celestial_frame
import sunpy.coordinates
import sunpy.map
from sunpy.data.sample import AIA_171_IMAGE
from astropy.coordinates import SkyCoord
import astropy.units as u
amap = sunpy.map.Map(AIA_171_IMAGE)
wcs_to_celestial_frame(amap.wcs)

<Helioprojective Frame (obstime=2011-06-07T06:33:02.770, rsun=696000000.0 m, observer=<HeliographicStonyhurst Coordinate (obstime=2011-06-07T06:33:02.770): (lon, lat, radius) in (deg, deg, m)
    (-0.00406308, 0.04787238, 1.51846026e+11)>)>

## 座標系変換が一番の肝なのでそこを試してみる
Hinode→Helioprojective
SWAN→Heliographic Stonyhurst


In [30]:
import pandas as pd
from sunpy.coordinates import frames
path = "../sot_sp/SOTSP_2010.csv"
sot_sp_df = pd.read_csv(path)
# c = Skycoord()
s=pd.Series(sot_sp_df.loc[0])
ll = [s["XCEN"]-s["FOVX"]//2,s["XCEN"]+s["FOVX"]//2]
ur =[s["YCEN"]-s["FOVY"]//2,s["YCEN"]+s["FOVY"]//2]
c=SkyCoord(ll*u.arcsec,ur*u.arcsec,frame = frames.Helioprojective,obstime= "2010-01-01",observer="earth")
c.transform_to(frames.HeliographicStonyhurst)

<SkyCoord (HeliographicStonyhurst: obstime=2010-01-01T00:00:00.000): (lon, lat, radius) in (deg, deg, AU)
    [(17.83207312, -33.35237352, 0.00465047),
     (37.45921616, -22.3977444 , 0.00465047)]>